# **Esquema de comunicación**

### **Menú**
* Codificacion Huffman
* Codificacion Shannon
* Codificacion LZW
* Codificacion RLE

### **Función principal para leer el archivo .txt**

In [1]:
def leer_archivo(nombre_archivo):

    try:
        with open(nombre_archivo, encoding="utf8") as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

### **Codificación Huffman**

In [2]:
def texto_a_binario(mensaje):
    binario = ''.join(format(ord(char), '08b') for char in mensaje)
    return binario

# El node del arbol
class Nodo:
    def __init__(self, char, freq, izq=None, der=None):
        self.char = char # Caracter.
        self.freq = freq # Frecuencia.
        self.izq = izq # Izquierda.
        self.der = der # Derecha.

def arbol(texto):
    freq = dict()
    for char in texto:
        if char not in freq:
            freq[char] = 0
        freq[char] += 1

    nodos = [Nodo(char, f) for char, f in freq.items()]
    while len(nodos) > 1:
        nodos = sorted(nodos, key=lambda x: x.freq)  # Ordena los nodos por frecuencia
        izq = nodos.pop(0)
        der = nodos.pop(0)
        nodo = Nodo(None, izq.freq + der.freq, izq, der)
        nodos.append(nodo)
    return nodos[0]  # Nodo raíz

def codificacion_HUFFMAN(nodo, codigo_binario="", diccionario=dict()):
    if nodo is None:
        return
    if nodo.char is not None:
        diccionario[nodo.char] = codigo_binario
    codificacion_HUFFMAN(nodo.izq, codigo_binario + "0", diccionario)
    codificacion_HUFFMAN(nodo.der, codigo_binario + "1", diccionario)
    return diccionario

def decodificacion_HUFFMAN(texto_codificado, nodo_raiz):
    texto = ""
    nodo_actual = nodo_raiz
    for bit in texto_codificado:
        if bit == '0':
            nodo_actual = nodo_actual.izq
        else:  # bit == '1'
            nodo_actual = nodo_actual.der
        if nodo_actual.char is not None:  # Es un nodo hoja
            texto += nodo_actual.char
            nodo_actual = nodo_raiz
    return texto

### **Codificación Shannon**

In [3]:
class Compresor: # Definimos una clase llamada "Compresor".
# Clase comprimida para almacenar diferentes parámetros de cadena.
    def __init__(self, caracter):
        self.original = caracter # Almacena el carácter original sin comprimir.
        self.contador = 0 # Se utiliza para llevar un registro del número de veces que aparece el carácter en la cadena comprimida.
        self.codigo = "" # Se utiliza para almacenar el código comprimido asociado al carácter.
        self.probabilidad = 0 # Se utiliza para almacenar la probabilidad de que el carácter aparezca en la cadena original.

# Esta función está destinada a ser utilizada para ordenar una lista de compresores según su probabilidad.
def obtener_probabilidad(compresores):
    return compresores.probabilidad

# Dividir probabilidades en orden utilizadas en la codificación shannon.
def divisor(probabilidad, puntero):
    # Calcula la diferencia entre la suma de las probabilidades en la parte izquierda del puntero y la suma de las probabilidades en la parte derecha del puntero. 
    diff = sum(probabilidad[:puntero+1]) - sum(probabilidad[puntero+1:])
    # Comprueba si la diferencia calculada es negativa, lo que implica que la división actual no es adecuada y debe intentar otra división.
    if diff < 0:
        # Llama recursivamente a la función divisor con el mismo conjunto de probabilidades, pero incrementa el puntero en 1.
        punto = divisor(probabilidad, puntero+1)
        diff_1 = sum(probabilidad[:punto]) - sum(probabilidad[punto:])
        diff_2 = sum(probabilidad[:punto+1]) - sum(probabilidad[punto+1:])
        # Compara las diferencias diff_1 y diff_2 y determina cuál de ellas es más pequeña en valor absoluto.
        if abs(diff_1) < abs(diff_2):
            return punto - 1
        return punto
    else:
        return puntero

# Se encarga de asignar códigos "0" y "1" a los compresores. 
def codificador(compresores, particion):
    if particion > 0: # Indica que todavía hay compresores que deben recibir códigos.
        parte_1 = compresores[:particion+1]
        for i in parte_1:
            i.codigo += '0' # Esto indica que estos compresores están en la parte izquierda de la partición.
        if len(parte_1) <= 1: # En este caso, retorna y no se realizan más divisiones.
            return
        codificador(parte_1, divisor(probabilidad=[i.probabilidad for i in parte_1], puntero=0))
        parte_2 = compresores[particion+1:]
        for i in parte_2:
            i.codigo += '1' # Esto indica que estos compresores están en la parte derecha de la partición.
        if len(parte_2) <= 1:
            return
        codificador(parte_2, divisor(probabilidad=[i.probabilidad for i in parte_2], puntero=0))
    elif particion == 0: # En caso de que la partición sea igual a cero, significa que todos los compresores se encuentran en la misma partición.
        parte_1 = compresores[:particion+1]
        for i in parte_1:
            i.codigo += '0'
        parte_2 = compresores[particion+1:]
        for i in parte_2:
            i.codigo += '1'

# Realiza el proceso de compresión de datos utilizando el algoritmo de Shannon-Fano.
def comprimir_datos(datos):
    procesados = [] # Crea una lista vacía llamada procesados para mantener un registro de los caracteres procesados.
    compresores = [] # Crea una lista vacía llamada compresores que almacenará objetos Compresor.
    longitud_total = len(datos)
    for i in range(len(datos)):
        if datos[i] not in procesados: # Verifica si el carácter actual no ha sido procesado previamente (no se encuentra en la lista procesados).
            procesados.append(datos[i]) # Agrega el carácter actual a la lista procesados para indicar que ha sido procesado.
            conteo = datos.count(datos[i]) # Cuenta cuántas veces aparece el carácter actual en la cadena de entrada y almacena el resultado en la variable conteo.
            var = conteo / longitud_total # Calcula la probabilidad del carácter actual dividiendo su conteo por la longitud total de la cadena.
            comp = Compresor(datos[i])
            comp.contador = conteo
            comp.probabilidad = var
            compresores.append(comp)
    compresores_ordenados = sorted(compresores, key=obtener_probabilidad, reverse=True) # Ordena la lista de compresores en orden descendente de probabilidad utilizando la función obtener_probabilidad como clave de ordenamiento. 
    particion = divisor(probabilidad=[i.probabilidad for i in compresores_ordenados], puntero=0)
    codificador(compresores_ordenados, particion)
    return compresores_ordenados


### **Códificación Lempel Ziv y Welch**

In [4]:
def comprimir_LZW(texto):
    diccionario = {chr(i): i for i in range(256)}  # Inicializar el diccionario con caracteres ASCII
    siguiente_codigo = 256
    resultado = []
    codigo_actual = ""

    for caracter in texto:
        codigo_actual += caracter
        if codigo_actual not in diccionario:
            diccionario[codigo_actual] = siguiente_codigo
            siguiente_codigo += 1
            resultado.append(diccionario[codigo_actual[:-1]])
            codigo_actual = caracter

    resultado.append(diccionario[codigo_actual])  # Agregar el último código

    return resultado

def descomprimir_LZW(datos_comprimidos):
    diccionario = {i: chr(i) for i in range(256)}  # Inicializar el diccionario con caracteres ASCII
    siguiente_codigo = 256
    resultado = []
    codigo_actual = chr(datos_comprimidos[0])
    resultado.append(codigo_actual)

    for codigo in datos_comprimidos[1:]:
        if codigo in diccionario:
            entrada = diccionario[codigo]
        elif codigo == siguiente_codigo:
            entrada = codigo_actual + codigo_actual[0]
        else:
            raise ValueError("Datos inválidos.")

        resultado.append(entrada)
        diccionario[siguiente_codigo] = codigo_actual + entrada[0]
        siguiente_codigo += 1
        codigo_actual = entrada

    print("Diccionario:")
    for clave, valor in diccionario.items():
        print(f"{clave}: {valor}")

    return ''.join(resultado)

### **Codificación Run-Length Encoding**

In [5]:
def codificacion_RLE(texto):
    texto_codificado = []
    conteo = 1

    for i in range(1, len(texto)):
        if texto[i] == texto[i - 1]:
            conteo += 1
        else:
            texto_codificado.append(texto[i - 1] + str(conteo))
            conteo = 1
    
    texto_codificado.append(texto[-1] + str(conteo))
    print( "Texto codificado: ",texto_codificado)

    return "".join(texto_codificado)

def decodificacion_RLE(texto_codificado):
    texto_decodificado = []
    i = 0

    while i < len(texto_codificado):
        char = texto_codificado[i]
        i += 1
        count_str = ""
        while i < len(texto_codificado) and texto_codificado[i].isdigit():
            count_str += texto_codificado[i]
            i += 1
        count = int(count_str)
        texto_decodificado.append(char * count)

    return "".join(texto_decodificado)

def diccionario_RLE(texto):
    conjunto = set(texto)
    print("Diccionario implícito de RLE:")
    for caracter in conjunto:
        print(f"'{caracter}' -> {caracter}")

## Menú

In [6]:
# Obtener la fuente de información (puedes leer un archivo o ingresar texto manualmente)
nombre_archivo_fuente_1 = "fuente 1.txt"
nombre_archivo_fuente_2 = "fuente 2.txt"
nombre_archivo_fuente_3 = "fuente 3.txt"
texto_original_1 = leer_archivo(nombre_archivo_fuente_1)
texto_original_2 = leer_archivo(nombre_archivo_fuente_2)
texto_original_3= leer_archivo(nombre_archivo_fuente_3)

# Menú para seleccionar el algoritmo de compresión
while True:
    print("|------------------------------------------|")
    print("| Seleccione un algoritmo de compresión:   |")
    print("|------------------------------------------|")
    print("| 1. Huffman                               |")
    print("| 2. Shannon-Fano                          |")
    print("| 3. LZW                                   |")
    print("| 4. RLE                                   |")
    print("| 0. Salir                                 |")
    print("|------------------------------------------|")
    opcion = int(input("Opción: "))
    if opcion == 0:
        print("")
        print("╔═══════════════════════════╗")
        print("║         Saliendo...       ║")
        print("╚═══════════════════════════╝")
        print("")
        break
    elif opcion == 1:
        print("")
        print("|------------------------|")
        print("|  Codificación Huffman  |")
        print("|------------------------|")
        print("")
        texto_binario = texto_a_binario(texto_original_1)
        raiz = arbol(texto_original_1)
        diccionario = codificacion_HUFFMAN(raiz)
        texto_codificado = "".join(diccionario[char] for char in texto_original_1)
        texto_decodificado = decodificacion_HUFFMAN(texto_codificado, raiz)
        print("Cadena binaria original:",texto_binario)
        print("Cadena binaria comprimida:",texto_codificado)
        print("")
        print("Mensaje enviado:",texto_original_1)
        print("Mensaje recibido:",texto_decodificado)
        print("")
        print("Lista de simbolos: ")
        print(diccionario)
        print("")
    elif opcion == 2:
        print("")
        print("|------------------------|")
        print("|  Codificación Shannon  |")
        print("|------------------------|")
        print("")
        print("Texto original:", texto_original_1)
        print("")
        datos_comprimidos = comprimir_datos(texto_original_1)
        for i in datos_comprimidos:
            print(f"Caracter: {i.original}, Código: {i.codigo}, Probabilidad: {i.probabilidad}")
        print("")
    elif opcion == 3:
        print("")
        print("|------------------------|")
        print("|    Codificación LZW    |")
        print("|------------------------|")
        print("")
        print("Texto original:\n")
        print(texto_original_2)
        datos = comprimir_LZW(texto_original_2)
        print("")
        print("Datos comprimidos:", datos)
        print("")
        texto_decomprimido = descomprimir_LZW(datos)
        print("")
        print("Texto recibido:\n")
        print(texto_decomprimido)
        print("")
    elif opcion == 4:
        print()
        print("|------------------------|")
        print("|    Codificación RLE    |")
        print("|------------------------|")
        print("")
        texto_codificado = codificacion_RLE(texto_original_3)
        texto_decodificado = decodificacion_RLE(texto_codificado)
        print("")
        print("Texto original:\n\n",texto_original_3)
        print("\n Texto codificado: \n", texto_codificado)
        print("\nTexto decodificado:\n", texto_decodificado)
        print("")
        diccionario_RLE(texto_original_3)
        print("")
    else:
        print("")
        print("*** Opción no válida. Inténtalo de nuevo. ***")
        print("")

|------------------------------------------|
| Seleccione un algoritmo de compresión:   |
|------------------------------------------|
| 1. Huffman                               |
| 2. Shannon-Fano                          |
| 3. LZW                                   |
| 4. RLE                                   |
| 0. Salir                                 |
|------------------------------------------|



|------------------------|
|  Codificación Huffman  |
|------------------------|

Cadena binaria original: 0100100001101111011011000110000100101100001000000110010101110011011101000110111100100000011001010111001100100000011101010110111000100000011011010110010101101110011100110110000101101010011001010010000001100100011001010010000001110000011100100111010101100101011000100110000100100000011100000110000101110010011000010010000001101100011000010010000001100011011011110110110101110101011011100110100101100011011000010110001101101001011011110110111000101110
Cadena binaria comprimida: 1101001001100010111101011110010001110110100111100100011110100101011110110001101000010111101110011110000000111110111110000100001000010111111011101111000011111100010111110101100110110010010101100101010110101110011001101010000

Mensaje enviado: Hola, esto es un mensaje de prueba para la comunicacion.
Mensaje recibido: Hola, esto es un mensaje de prueba para la comunicacion.

Lista de simbolos: 
{'d': '00000', 'b': '